In [1]:
%load_ext autoreload
%autoreload 2
import logging
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import duckdb
import pandas as pd

from planter.database.utils.duckdb_utils import (
    extract_representative_sequences, 
    create_duckdb,
    merge_duckdbs,
    validate_duckdb_schema,
    update_clusters
)

In [2]:


samples = ['SRR12068547', 'SRR12068548', 'SRR12068549', 'SRR12068550', 'SRR12068551', 'SRR12068552']
outdir = Path('/mnt/data4/recombia.planter')
cluster_path = '/mnt/data4/planter_outputs/tmp/newClusterDB.tsv'

# 1. Create duckdbs for Mesoplasma samples
for sample in samples:
    duckdb_path = outdir / f'{sample}/{sample}.duckdb'
    if duckdb_path.exists():
        # remove the duckdb file
        duckdb_path.unlink()
    create_duckdb(
        sample_id=sample,
        outdir=outdir,
        duckdb_out=outdir / f'{sample}/{sample}.duckdb'
    )

# 2. Merge all duckdbs into a master duckdb
merge_duckdbs(
    duckdb_paths=[outdir / f'{sample}/{sample}.duckdb' for sample in samples],
    master_db_path=outdir / 'master.duckdb',
    schema_sql_path=Path('../planter/database/schema/migrations/004_add_gene_protein_map.sql'),
    upgrade_schema=True,
    target_schema_version=None
)

# 3. Validate the master duckdb
validate_duckdb_schema(outdir / 'master.duckdb')

# 4. Add cluster info to the master duckdb
update_clusters(
    db_path=outdir / 'master.duckdb',
    tsv_path=cluster_path,
    backup_first=True
)


Validating database: /mnt/data4/recombia.planter/master.duckdb

=== Tables in database ===
Table: annotations - 2049722 rows
Table: clusters - 428497 rows
Table: cluster_members - 433791 rows
Table: ec_numbers - 571387 rows
Table: expression - 3091553 rows
Table: gene_protein_map - 745417 rows
Table: go_terms - 74845016 rows
Table: kegg_info - 1626 rows
Table: schema_version - 0 rows
Table: sequences - 2489980 rows
Table: sra_metadata - 100 rows

=== Schema for each table ===

Schema for annotations:
  seqhash_id (VARCHAR) PRIMARY KEY
  seed_ortholog (VARCHAR)
  evalue (DOUBLE)
  score (DOUBLE)
  eggnog_ogs (VARCHAR)
  max_annot_lvl (VARCHAR)
  cog_category (VARCHAR)
  description (VARCHAR)
  preferred_name (VARCHAR)
  sample_id (VARCHAR)

Schema for clusters:
  cluster_id (VARCHAR) PRIMARY KEY
  representative_seqhash_id (VARCHAR)
  size (INTEGER)

Schema for cluster_members:
  seqhash_id (VARCHAR) PRIMARY KEY
  cluster_id (VARCHAR)

Schema for ec_numbers:
  seqhash_id (VARCHAR) PRIMA

Starting cluster update for database: /mnt/data4/recombia.planter/master.duckdb
Using clustering data from: /mnt/data4/planter_outputs/tmp/newClusterDB.tsv
Duplicate handling strategy: replace
Creating backup at: /mnt/data4/recombia.planter/master.duckdb.backup


Annotated proteins: 622261
Genes of annotated proteins: 622261
Genes with both annotation and expression: 622259

=== Cluster Data ===
Total clusters: 428497
Average cluster size: 1.01
Largest cluster size: 11
Unique sequences in clusters: 433791
Total cluster membership records: 433791

Top 3 largest clusters:
  Cluster v1_DLS_929072aaf5bbeda34f7d58ac3e0d9fedd11ad3818a914aeb4fd7bb3e0001eaaf.p1: 11 members
  Cluster v1_DLS_b6cf04a60dfba05cfefdf7b30e5096b124df4862e8ca750b224ce458186859b2.p1: 11 members
  Cluster v1_DLS_9d3fd01247bdb20571289a7f0b4330da0fbabceb59cfbb805bbba9dd26ea40dc.p1: 10 members


KeyboardInterrupt: 

In [7]:
# merge_duckdbs(
#     duckdb_paths=[outdir / f'{sample}/{sample}.duckdb' for sample in samples],
#     master_db_path=outdir / 'master.duckdb.backup',
#     schema_sql_path=Path('../planter/database/schema/migrations/004_add_gene_protein_map.sql'),
#     upgrade_schema=True,
#     target_schema_version=None
# )

In [3]:
outdir = Path('/mnt/data4/recombia.planter')
# cluster_path = '/mnt/data4/planter_outputs/tmp/newClusterDB.tsv'
cluster_path = '/tmp/newClusterDB.tsv'

update_clusters(
    db_path=outdir / 'master.duckdb',
    tsv_path=cluster_path,
    backup_first=True
)

Starting cluster update for database: /mnt/data4/recombia.planter/master.duckdb
Starting cluster update for database: /mnt/data4/recombia.planter/master.duckdb
Using clustering data from: /tmp/newClusterDB.tsv
Using clustering data from: /tmp/newClusterDB.tsv
Duplicate handling strategy: replace
Duplicate handling strategy: replace
Creating backup at: /mnt/data4/recombia.planter/master.duckdb.backup
Creating backup at: /mnt/data4/recombia.planter/master.duckdb.backup
Beginning database transaction
Beginning database transaction
Loading cluster data from TSV...
Loading cluster data from TSV...
Loaded 434601 entries from clustering TSV
Loaded 434601 entries from clustering TSV
Strategy is 'replace': Dropping existing cluster tables...
Strategy is 'replace': Dropping existing cluster tables...
Recreating cluster tables...
Recreating cluster tables...
Identifying missing sequences...
Identifying missing sequences...
Found 405 unique sequences missing from database
Found 405 unique sequence

'/mnt/data4/recombia.planter/master.duckdb.backup'

In [15]:
clusters = pd.read_csv(cluster_path, sep='\t', header=None, names=['cluster_id', 'seqhash_id'])
clusters[clusters['cluster_id'] == 'v1_DLS_388eb9d599d8964707f5e87139888647832fde5f2994928ba82de8076adf5010.p1']

,cluster_id,seqhash_id
310973,v1_DLS_388eb9d599d8964707f5e87139888647832fde5...,v1_DLS_388eb9d599d8964707f5e87139888647832fde5...


In [4]:
def debug_specific_seqhash(
    db_path: Union[str, Path],
    tsv_path: Union[str, Path],
    problem_seqhash: str = "v1_DLS_388eb9d599d8964707f5e87139888647832fde5f2994928ba82de8076adf5010.p1"
):
    """
    Debug a specific seqhash ID that's causing constraint violations.
    
    Args:
        db_path: Path to the DuckDB database file
        tsv_path: Path to the TSV file containing cluster information
        problem_seqhash: The specific sequence hash ID to investigate
    """
    db_path = str(db_path)
    tsv_path = str(tsv_path)
    
    print(f"Debugging seqhash ID: {problem_seqhash}")
    
    con = duckdb.connect(db_path)
    
    try:
        print("\n== Database State ==")
        
        # Check if the ID exists in the sequences table
        exists_in_sequences = con.execute(
            f"SELECT COUNT(*) FROM sequences WHERE seqhash_id = '{problem_seqhash}'"
        ).fetchone()[0]
        print(f"ID exists in sequences table: {exists_in_sequences > 0}")
        
        # Check if it exists as a cluster_id
        try:
            exists_as_cluster = con.execute(
                f"SELECT COUNT(*) FROM clusters WHERE cluster_id = '{problem_seqhash}'"
            ).fetchone()[0]
            print(f"ID exists as a cluster_id: {exists_as_cluster > 0}")
        except:
            print("clusters table doesn't exist or couldn't be queried")
        
        # Check if it exists in cluster_members
        try:
            exists_as_member = con.execute(
                f"SELECT COUNT(*) FROM cluster_members WHERE seqhash_id = '{problem_seqhash}'"
            ).fetchone()[0]
            print(f"ID exists in cluster_members table: {exists_as_member > 0}")
        except:
            print("cluster_members table doesn't exist or couldn't be queried")
        
        print("\n== TSV File Analysis ==")
        
        # Load the TSV temporarily to analyze it
        con.execute(f"""
            CREATE TEMP TABLE clustering_analysis AS
            SELECT 
                column0 AS representative_seqhash_id,
                column1 AS seqhash_id
            FROM read_csv_auto('{tsv_path}', sep='\t', header=FALSE)
        """)
        
        # Check occurrences of this ID in the TSV
        as_rep = con.execute(
            f"SELECT COUNT(*) FROM clustering_analysis WHERE representative_seqhash_id = '{problem_seqhash}'"
        ).fetchone()[0]
        as_seq = con.execute(
            f"SELECT COUNT(*) FROM clustering_analysis WHERE seqhash_id = '{problem_seqhash}'"
        ).fetchone()[0]
        
        print(f"ID appears as a representative in TSV: {as_rep} times")
        print(f"ID appears as a sequence in TSV: {as_seq} times")
        
        # If it appears as a sequence, show which cluster it belongs to
        if as_seq > 0:
            rep_id = con.execute(
                f"SELECT representative_seqhash_id FROM clustering_analysis WHERE seqhash_id = '{problem_seqhash}' LIMIT 1"
            ).fetchone()[0]
            print(f"This ID clusters with representative: {rep_id}")
        
        # Check for circular references
        if as_rep > 0 and as_seq > 0:
            print("\nPOTENTIAL CIRCULAR REFERENCE DETECTED!")
            print("This ID is both a representative sequence and a member of another cluster.")
            
            # Get details of the circular reference
            circle_info = con.execute(f"""
                SELECT c1.representative_seqhash_id as rep1, c1.seqhash_id as seq1, 
                       c2.representative_seqhash_id as rep2
                FROM clustering_analysis c1
                JOIN clustering_analysis c2 ON c1.seqhash_id = c2.representative_seqhash_id
                WHERE c1.seqhash_id = '{problem_seqhash}'
                LIMIT 1
            """).fetchall()
            
            if circle_info:
                print(f"Circular reference chain: {circle_info[0][0]} -> {circle_info[0][1]} -> {circle_info[0][2]}")
        
        print("\n== Insertion Simulation ==")
        
        # Try to simulate the insertion that might fail
        print("Simulating the operations that might cause the constraint violation...")
        
        # First check which table the constraint might be violated in
        try:
            if exists_as_member > 0:
                print("This ID already exists in cluster_members table, insertion would violate PRIMARY KEY constraint")
            elif exists_as_cluster > 0:
                print("This ID already exists in clusters table, insertion would violate PRIMARY KEY constraint")
            else:
                print("ID doesn't exist in clusters or cluster_members tables yet.")
                
                # Check if it would be inserted based on the filtering criteria
                would_be_inserted = con.execute(f"""
                    SELECT COUNT(*) FROM clustering_analysis ca
                    WHERE (ca.representative_seqhash_id = '{problem_seqhash}' OR ca.seqhash_id = '{problem_seqhash}')
                    AND EXISTS (SELECT 1 FROM sequences s WHERE s.seqhash_id = ca.representative_seqhash_id)
                    AND EXISTS (SELECT 1 FROM sequences s WHERE s.seqhash_id = ca.seqhash_id)
                """).fetchone()[0]
                
                print(f"ID would pass validation filters: {would_be_inserted > 0}")
        except Exception as e:
            print(f"Error during simulation: {str(e)}")
            
    except Exception as e:
        print(f"Error during debugging: {str(e)}")
    finally:
        try:
            con.execute("DROP TABLE IF EXISTS clustering_analysis")
        except:
            pass
        con.close()
    
    print("\n== Recommendation ==")
    print("Based on the analysis, consider using 'handle_duplicates=\"replace\"' to completely")
    print("refresh the cluster tables, which should avoid any constraint violations.")

outdir = Path('/mnt/data4/recombia.planter')
cluster_path = '/mnt/data4/planter_outputs/tmp/newClusterDB.tsv'
debug_specific_seqhash(
    db_path=outdir / 'master.duckdb',
    tsv_path=cluster_path,
    problem_seqhash="v1_DLS_a2fb01ed3b3a4cd07a6f0b5063937e2c85390e49ef85f32b295474a9c5c7722f.p1"
)


Debugging seqhash ID: v1_DLS_a2fb01ed3b3a4cd07a6f0b5063937e2c85390e49ef85f32b295474a9c5c7722f.p1

== Database State ==
ID exists in sequences table: False
ID exists as a cluster_id: False
ID exists in cluster_members table: False

== TSV File Analysis ==
ID appears as a representative in TSV: 1 times
ID appears as a sequence in TSV: 2 times
This ID clusters with representative: v1_DLS_ffddc1838d79d4e7db61ea10a6b5eabe6699f0c588ee0033c19083b741b2141b.p1

POTENTIAL CIRCULAR REFERENCE DETECTED!
This ID is both a representative sequence and a member of another cluster.
Circular reference chain: v1_DLS_ffddc1838d79d4e7db61ea10a6b5eabe6699f0c588ee0033c19083b741b2141b.p1 -> v1_DLS_a2fb01ed3b3a4cd07a6f0b5063937e2c85390e49ef85f32b295474a9c5c7722f.p1 -> v1_DLS_a2fb01ed3b3a4cd07a6f0b5063937e2c85390e49ef85f32b295474a9c5c7722f.p1

== Insertion Simulation ==
Simulating the operations that might cause the constraint violation...
ID doesn't exist in clusters or cluster_members tables yet.
ID would pass

In [3]:
def check_tables_in_db(db_path):
    """
    Check which tables exist in a DuckDB database.
    
    Args:
        db_path: Path to the DuckDB file
        
    Returns:
        Dictionary of table names and their row counts
    """
    db_path = Path(db_path)
    
    if not db_path.exists():
        print(f"Database file does not exist: {db_path}")
        return {}
    
    try:
        with duckdb.connect(str(db_path)) as conn:
            # Get all table names
            tables = conn.execute(
                "SELECT name FROM sqlite_master WHERE type='table'"
            ).fetchall()
            
            # Create a dictionary to store table info
            table_info = {}
            
            # For each table, get row count and column info
            for table_row in tables:
                table_name = table_row[0]
                
                # Get row count
                try:
                    row_count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
                except Exception as e:
                    row_count = f"Error: {str(e)}"
                
                # Get column info
                try:
                    columns = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
                    column_names = [col[1] for col in columns]
                except Exception as e:
                    column_names = [f"Error: {str(e)}"]
                
                # Store table info
                table_info[table_name] = {
                    "row_count": row_count,
                    "columns": column_names
                }
            
            return table_info
    except Exception as e:
        print(f"Error connecting to database: {str(e)}")
        return {}
    
# db_path = "/mnt/data4/recombia.planter/SRR12068547/SRR12068547.duckdb"
db_path = '/mnt/data4/recombia.planter/master.duckdb'
check_tables_in_db(db_path)

{'annotations': {'row_count': 2049722,
  'columns': ['seqhash_id',
   'seed_ortholog',
   'evalue',
   'score',
   'eggnog_ogs',
   'max_annot_lvl',
   'cog_category',
   'description',
   'preferred_name',
   'sample_id']},
 'clusters': {'row_count': 433785,
  'columns': ['cluster_id', 'representative_seqhash_id', 'size']},
 'cluster_members': {'row_count': 2489980,
  'columns': ['seqhash_id', 'cluster_id']},
 'ec_numbers': {'row_count': 571387, 'columns': ['seqhash_id', 'ec_number']},
 'expression': {'row_count': 3091553,
  'columns': ['gene_seqhash_id',
   'sample_id',
   'tpm',
   'num_reads',
   'effective_length']},
 'gene_protein_map': {'row_count': 744572,
  'columns': ['gene_seqhash_id', 'protein_seqhash_id']},
 'go_terms': {'row_count': 74845016, 'columns': ['seqhash_id', 'go_term']},
 'kegg_info': {'row_count': 0,
  'columns': ['seqhash_id',
   'kegg_ko',
   'kegg_pathway',
   'kegg_module',
   'kegg_reaction',
   'kegg_rclass']},
 'schema_version': {'row_count': 0,
  'colum

In [85]:
db_path

'/mnt/data4/recombia.planter/master.duckdb'

In [ ]:
cluster_path = '/mnt/data4/planter_outputs/tmp/newClusterDB.tsv'
from planter.database.utils.duckdb_utils import (
    update_clusters
)

update_clusters(
    db_path=db_path,
    tsv_path=cluster_path,
    backup_first=True
)


In [ ]:
import pandas as pd
clusters = pd.read_csv(cluster_path, sep='\t', header=None, names=['cluster_id', 'seqhash_id'])
clusters[clusters['cluster_id'] == 'v1_DLS_42f03501d692b370647a4bac7059434916aa1d21968fca917aa214f32f2ced17.p1'].shape

(35, 2)

In [74]:
def debug_foreign_key_issues(
    db_path: Union[str, Path],
    problematic_id: str = "v1_DLS_19696ccb77b302cd0cdff9203ea61ab6436f11d2972fe64be97b00a5961373da.p1"
) -> None:
    """
    Debug foreign key constraint issues with a specific cluster ID.
    
    Args:
        db_path: Path to the DuckDB database
        problematic_id: The specific ID causing constraint issues
    """
    db_path = str(db_path)
    con = duckdb.connect(db_path)
    
    try:
        print(f"Debugging foreign key constraints for ID: {problematic_id}")
        
        # 1. First, check if this ID exists in the clusters table
        cluster_exists = con.execute(
            f"SELECT COUNT(*) FROM clusters WHERE cluster_id = '{problematic_id}'"
        ).fetchone()[0]
        print(f"ID exists in clusters table: {cluster_exists > 0}")
        
        # 2. Get all tables in the database
        tables = con.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
        print(f"Found {len(tables)} tables in the database")
        
        # 3. Check each table to see if it references clusters
        references = []
        for table in tables:
            table_name = table[0]
            if table_name == 'clusters':
                continue
                
            try:
                # Check if the table has foreign keys to clusters
                fk_info = con.execute(f"PRAGMA foreign_key_list({table_name})").fetchall()
                for fk in fk_info:
                    if fk[2] == 'clusters':
                        ref_table = table_name
                        ref_col = fk[3]  # Referenced column in clusters
                        local_col = fk[4]  # Local column in this table
                        references.append((ref_table, local_col, ref_col))
                        print(f"Table {ref_table} references clusters.{ref_col} via {local_col}")
            except Exception as e:
                print(f"Error checking foreign keys for table {table_name}: {str(e)}")
        
        # 4. For each table that references clusters, check if our problematic ID is referenced
        found_references = False
        for ref_table, local_col, ref_col in references:
            try:
                if ref_col == 'cluster_id':
                    # Check if this table references our problematic ID
                    ref_count = con.execute(
                        f"SELECT COUNT(*) FROM {ref_table} WHERE {local_col} = '{problematic_id}'"
                    ).fetchone()[0]
                    
                    if ref_count > 0:
                        found_references = True
                        print(f"FOUND REFERENCE: Table {ref_table} has {ref_count} rows referencing the problematic ID")
                        
                        # Get sample rows to understand the reference
                        sample_rows = con.execute(
                            f"SELECT * FROM {ref_table} WHERE {local_col} = '{problematic_id}' LIMIT 3"
                        ).fetchall()
                        print(f"Sample rows from {ref_table}:")
                        for row in sample_rows:
                            print(f"  {row}")
            except Exception as e:
                print(f"Error checking references in table {ref_table}: {str(e)}")
        
        if not found_references:
            print("No explicit references to the problematic ID were found.")
            print("This might indicate a schema issue or a constraint that's not properly detected.")
            
        # 5. Check if the ID might be in a different format or location
        print("\nChecking for ID in different formats:")
        for table in tables:
            table_name = table[0]
            try:
                # Get column names for this table
                columns = con.execute(f"PRAGMA table_info({table_name})").fetchall()
                column_names = [col[1] for col in columns]
                
                # Check each column that might contain IDs
                for col in column_names:
                    if "id" in col.lower() or "hash" in col.lower() or "key" in col.lower():
                        # Look for exact match
                        exact_match = con.execute(
                            f"SELECT COUNT(*) FROM {table_name} WHERE {col} = '{problematic_id}'"
                        ).fetchone()[0]
                        
                        # Look for partial match
                        partial_match = con.execute(
                            f"SELECT COUNT(*) FROM {table_name} WHERE {col} LIKE '%{problematic_id}%'"
                        ).fetchone()[0]
                        
                        if exact_match > 0:
                            print(f"Table {table_name}.{col} contains exact match: {exact_match} rows")
                        elif partial_match > 0:
                            print(f"Table {table_name}.{col} contains partial match: {partial_match} rows")
            except Exception as e:
                print(f"Error checking table {table_name} columns: {str(e)}")
                
        # 6. Check if the clusters table has the expected structure
        print("\nClusters table structure:")
        cluster_cols = con.execute("PRAGMA table_info(clusters)").fetchall()
        for col in cluster_cols:
            print(f"  {col}")
            
        print("\nCluster_members table structure:")
        member_cols = con.execute("PRAGMA table_info(cluster_members)").fetchall()
        for col in member_cols:
            print(f"  {col}")
        
    except Exception as e:
        print(f"Error during debugging: {str(e)}")
    finally:
        con.close()

debug_foreign_key_issues(db_path)

Debugging foreign key constraints for ID: v1_DLS_19696ccb77b302cd0cdff9203ea61ab6436f11d2972fe64be97b00a5961373da.p1
ID exists in clusters table: True
Found 12 tables in the database
Error checking foreign keys for table annotations: Catalog Error: Pragma Function with name foreign_key_list does not exist!
Did you mean "force_checkpoint"?
Error checking foreign keys for table cluster_members: Catalog Error: Pragma Function with name foreign_key_list does not exist!
Did you mean "force_checkpoint"?
Error checking foreign keys for table ec_numbers: Catalog Error: Pragma Function with name foreign_key_list does not exist!
Did you mean "force_checkpoint"?
Error checking foreign keys for table expression: Catalog Error: Pragma Function with name foreign_key_list does not exist!
Did you mean "force_checkpoint"?
Error checking foreign keys for table expression_backup: Catalog Error: Pragma Function with name foreign_key_list does not exist!
Did you mean "force_checkpoint"?
Error checking fore

In [6]:
def check_sequence_id(
    db_path: Union[str, Path],
    tsv_path: Union[str, Path],
    seqhash_id: str = "v1_DLS_a0a80e8508b66a50baba10818dc3b384cc13f0efb454dde8fc2c0dba1c936b19.p1"
) -> None:
    """
    Check if a specific sequence ID exists in the database and clustering TSV.
    
    Args:
        db_path: Path to the DuckDB database
        tsv_path: Path to the TSV file with cluster info
        seqhash_id: The specific sequence ID to check
    """
    db_path = str(db_path)
    tsv_path = str(tsv_path)
    
    print(f"Checking for sequence ID: {seqhash_id}")
    
    # Check database tables
    con = duckdb.connect(db_path)
    try:
        # Check if this ID exists in sequences table
        exists_in_sequences = con.execute(
            f"SELECT COUNT(*) FROM sequences WHERE seqhash_id = '{seqhash_id}'"
        ).fetchone()[0]
        
        print(f"ID exists in sequences table: {exists_in_sequences > 0}")
        
        if exists_in_sequences > 0:
            # Get details about this sequence
            seq_details = con.execute(
                f"SELECT * FROM sequences WHERE seqhash_id = '{seqhash_id}'"
            ).fetchone()
            print(f"Sequence details: {seq_details}")
        
        # Check TSV file for this ID
        con.execute(f"""
            CREATE TEMP TABLE clustering_check AS
            SELECT 
                column0 AS representative_seqhash_id,
                column1 AS seqhash_id
            FROM read_csv_auto('{tsv_path}', sep='\t', header=FALSE)
        """)
        
        # Check if ID appears as a sequence in clusters
        as_sequence = con.execute(
            f"SELECT COUNT(*) FROM clustering_check WHERE seqhash_id = '{seqhash_id}'"
        ).fetchone()[0]
        
        # Check if ID appears as a representative
        as_representative = con.execute(
            f"SELECT COUNT(*) FROM clustering_check WHERE representative_seqhash_id = '{seqhash_id}'"
        ).fetchone()[0]
        
        print(f"ID appears in TSV as sequence: {as_sequence > 0} ({as_sequence} times)")
        print(f"ID appears in TSV as representative: {as_representative > 0} ({as_representative} times)")
        
        # Get sample of problematic entries from TSV
        if as_sequence > 0:
            sample = con.execute(
                f"SELECT * FROM clustering_check WHERE seqhash_id = '{seqhash_id}' LIMIT 3"
            ).fetchall()
            print(f"Sample entries from TSV where ID is sequence:")
            for entry in sample:
                print(f"  {entry}")
        
        # Count total sequences in TSV not in sequences table
        missing_seqs = con.execute("""
            SELECT COUNT(DISTINCT tc.seqhash_id) 
            FROM clustering_check tc
            LEFT JOIN sequences s ON tc.seqhash_id = s.seqhash_id
            WHERE s.seqhash_id IS NULL
        """).fetchone()[0]
        
        print(f"Total sequences in TSV missing from sequences table: {missing_seqs}")
        
        # Get sample of missing sequences
        if missing_seqs > 0:
            sample_missing = con.execute("""
                SELECT DISTINCT tc.seqhash_id 
                FROM clustering_check tc
                LEFT JOIN sequences s ON tc.seqhash_id = s.seqhash_id
                WHERE s.seqhash_id IS NULL
                LIMIT 10
            """).fetchall()
            
            print("Sample of missing sequences:")
            for i, (seq_id,) in enumerate(sample_missing, 1):
                print(f"  {i}. {seq_id}")
    
    except Exception as e:
        print(f"Error during check: {str(e)}")
    finally:
        try:
            con.execute("DROP TABLE IF EXISTS clustering_check")
        except:
            pass
        con.close()

check_sequence_id(db_path, cluster_path)

Checking for sequence ID: v1_DLS_a0a80e8508b66a50baba10818dc3b384cc13f0efb454dde8fc2c0dba1c936b19.p1
ID exists in sequences table: False
ID appears in TSV as sequence: True (1 times)
ID appears in TSV as representative: False (0 times)
Sample entries from TSV where ID is sequence:
  ('v1_DLS_76400afa768372fd4118749f12b4252d972ea5024d4a582eaae1406cdf2c409b.p1', 'v1_DLS_a0a80e8508b66a50baba10818dc3b384cc13f0efb454dde8fc2c0dba1c936b19.p1')
Total sequences in TSV missing from sequences table: 2585
Sample of missing sequences:
  1. v1_DLS_0aa8c2e83469e1ffc35fcd62036e912dc065345e35af76618ad10d60c93254a3.p1
  2. v1_DLS_535ef37f0da2d881934117e345d4b080562c095bf4261380f2e186568bf6ce93.p1
  3. v1_DLS_ad38a37bc806cbc7f948fa82d627b090ebd1dd34198b0690c51207a76b2c64ed.p1
  4. v1_DLS_5e3b94322da686cedfffbf7ad8b27321fb217fb3036f886eb871e70d16898a1a.p1
  5. v1_DLS_f973093a0658b4dddb4aefcece0e9bf8368a87a5b3708d8ee2a50330bdb478fb.p2
  6. v1_DLS_b4948245d0398f054a1a41528136e3577c125a65504f2009ad6ea667c89e